In [7]:
import os
import glob
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from scipy.signal import resample
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# =========================================================
# 0. 設定エリア
# =========================================================
BASE_DIR = r"C:\Users\fujiw\OneDrive\デスクトップ\ECG_ResNet"

STAGE2_DIR = os.path.join(BASE_DIR, "stage2")       # NPYフォルダ
CSV_DIR    = os.path.join(BASE_DIR, "train_csvs")   # CSVフォルダ
TRAIN_META = os.path.join(BASE_DIR, "train.csv")    # train.csv
SAVE_DIR   = os.path.join(BASE_DIR, "models")       # 保存先

# ★モデルを大きくしたので、VRAM不足を防ぐためにバッチサイズを少し下げます
BATCH_SIZE = 32
EPOCHS = 200
PATIENCE = 20
LR = 1e-3

# =========================================================
# 1. Dataset Class (正規化を追加)
# =========================================================
class ECGDatasetRam(Dataset):
    def __init__(self, df, npy_dir, csv_dir, target_len=5000):
        self.target_len = target_len
        self.samples = [] 
        
        if not os.path.exists(npy_dir):
            raise FileNotFoundError(f"Directory not found: {npy_dir}")
            
        target_ids = set(df['id'].astype(str).tolist())
        file_list = []
        all_files = glob.glob(os.path.join(npy_dir, "*.npy"))
        
        print(f"Scanning files in {npy_dir}...")
        for fpath in all_files:
            fname = os.path.basename(fpath)
            file_id = fname.split('-')[0]
            if file_id in target_ids:
                file_list.append((fpath, file_id))
        
        print(f"Found {len(file_list)} files. Loading into RAM...")

        for fpath, sample_id in tqdm(file_list, desc="Pre-loading"):
            processed = self.process_one_file(fpath, sample_id, csv_dir)
            if processed is not None:
                self.samples.append(processed)
                
        print(f"Loaded {len(self.samples)} samples.")

    def process_one_file(self, npy_path, sample_id, csv_dir):
        try:
            # Input
            data = np.load(npy_path)
            data = np.nan_to_num(data, nan=0.0)
            original_len = data.shape[1]
            if data.shape[0] != 13: return None

            reconstructed = np.zeros((12, original_len), dtype=np.float32)
            for i in range(4):
                sig_row = data[i]
                id_row = data[9+i]
                unique_ids = np.unique(id_row)
                for uid in unique_ids:
                    if 0 <= uid <= 11:
                        mask_ch = (id_row == uid)
                        reconstructed[int(uid), mask_ch] = sig_row[mask_ch]
            
            # Target
            csv_path = os.path.join(csv_dir, f"{sample_id}.csv")
            if not os.path.exists(csv_path): return None

            target_df = pd.read_csv(csv_path)
            target_vals = target_df.values.T 
            mask_data = (~np.isnan(target_vals)).astype(np.float32)
            target_data = np.nan_to_num(target_vals, nan=0.0)
            
            # Resample
            if reconstructed.shape[1] != self.target_len:
                input_final = resample(reconstructed, self.target_len, axis=1)
            else:
                input_final = reconstructed
                
            if target_data.shape[1] != self.target_len:
                target_final = resample(target_data, self.target_len, axis=1)
                mask_final = resample(mask_data, self.target_len, axis=1)
            else:
                target_final = target_data
                mask_final = mask_data

            # ★【重要】正規化 (Standardization)
            # モデルを巨大化する場合、入力値のスケールが揃っていないと学習が発散します。
            # データそのものは変えませんが、平均0・分散1に揃えます。
            mean = np.mean(input_final, axis=1, keepdims=True)
            std = np.std(input_final, axis=1, keepdims=True) + 1e-6
            input_final = (input_final - mean) / std
            
            return (np.nan_to_num(input_final).astype(np.float32), 
                    np.nan_to_num(target_final).astype(np.float32), 
                    (mask_final > 0.5).astype(np.float32), 
                    original_len)
        except:
            return None

    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        in_arr, tgt_arr, msk_arr, length = self.samples[idx]
        return (torch.from_numpy(in_arr), torch.from_numpy(tgt_arr), 
                torch.from_numpy(msk_arr), torch.tensor(length, dtype=torch.long))

# =========================================================
# 2. Model (Large Version: チャネル数倍増・Dropout追加)
# =========================================================
class ResNet1d_UNet_Large(nn.Module):
    def __init__(self):
        super().__init__()
        
        # ★チャネル数を大幅に増加 (12 -> 128 スタート)
        # Encoder
        self.enc1 = nn.Sequential(nn.Conv1d(12, 128, 7, 2, 3), nn.BatchNorm1d(128), nn.ReLU())
        self.enc2 = nn.Sequential(nn.Conv1d(128, 256, 3, 2, 1), nn.BatchNorm1d(256), nn.ReLU())
        self.enc3 = nn.Sequential(nn.Conv1d(256, 512, 3, 2, 1), nn.BatchNorm1d(512), nn.ReLU())
        self.enc4 = nn.Sequential(nn.Conv1d(512, 1024, 3, 2, 1), nn.BatchNorm1d(1024), nn.ReLU())
        
        # ★Dropout (過学習防止)
        self.dropout = nn.Dropout(0.3)

        # Decoder
        # skip接続があるため入力チャネル数は (下層の出力 + Encoderの出力)
        self.dec4 = nn.Sequential(nn.Conv1d(1024 + 512, 512, 3, 1, 1), nn.BatchNorm1d(512), nn.ReLU())
        self.dec3 = nn.Sequential(nn.Conv1d(512 + 256, 256, 3, 1, 1), nn.BatchNorm1d(256), nn.ReLU())
        self.dec2 = nn.Sequential(nn.Conv1d(256 + 128, 128, 3, 1, 1), nn.BatchNorm1d(128), nn.ReLU())
        
        # 最後のアップサンプル層
        self.dec1 = nn.Sequential(nn.Conv1d(128, 64, 3, 1, 1), nn.BatchNorm1d(64), nn.ReLU())
        
        # 出力層 (64 -> 12)
        self.final = nn.Conv1d(64, 12, 1)

    def forward(self, x):
        # Encoder
        e1 = self.enc1(x)
        e2 = self.enc2(e1)
        e3 = self.enc3(e2)
        e4 = self.enc4(e3)
        
        # Bottleneck Dropout
        e4 = self.dropout(e4)
        
        # Decoder
        d4 = torch.cat([torch.nn.functional.interpolate(e4, size=e3.shape[2]), e3], dim=1)
        d4 = self.dec4(d4)
        
        d3 = torch.cat([torch.nn.functional.interpolate(d4, size=e2.shape[2]), e2], dim=1)
        d3 = self.dec3(d3)
        
        d2 = torch.cat([torch.nn.functional.interpolate(d3, size=e1.shape[2]), e1], dim=1)
        d2 = self.dec2(d2)
        
        # Final Upsample (No Skip Connection here to match dimensionality)
        d1 = torch.nn.functional.interpolate(d2, size=x.shape[2])
        d1 = self.dec1(d1)
        
        out = self.final(d1)
        
        return x + out

# =========================================================
# 3. Training Loop (勾配クリッピング追加)
# =========================================================
def run_training():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using Device: {device}")
    
    if not os.path.exists(SAVE_DIR): os.makedirs(SAVE_DIR)
    if not os.path.exists(TRAIN_META): return

    df = pd.read_csv(TRAIN_META)
    unique_ids = df['id'].unique()
    train_ids, val_ids = train_test_split(unique_ids, test_size=0.1, random_state=42)
    
    print("Dataset Init...")
    train_ds = ECGDatasetRam(df[df['id'].isin(train_ids)], STAGE2_DIR, CSV_DIR)
    val_ds = ECGDatasetRam(df[df['id'].isin(val_ids)], STAGE2_DIR, CSV_DIR)
    
    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
    val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
    
    # ★巨大モデルを使用
    model = ResNet1d_UNet_Large().to(device)
    optimizer = optim.Adam(model.parameters(), lr=LR)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=7)
    
    use_amp = torch.cuda.is_available()
    scaler = torch.amp.GradScaler('cuda') if use_amp else None
    criterion_raw = nn.MSELoss(reduction='none')

    best_loss = float('inf')
    patience_counter = 0
    
    print("Training Started (Large Model)...")
    for epoch in range(EPOCHS):
        current_epoch = epoch + 1
        print(f"\n{'='*20} Epoch {current_epoch}/{EPOCHS} {'='*20}")
        
        # Train
        model.train()
        train_loss = 0
        pbar = tqdm(train_loader, desc=f"Training   (Epoch {current_epoch})")
        
        for inputs, targets, masks, _ in pbar:
            inputs, targets, masks = inputs.to(device), targets.to(device), masks.to(device)
            optimizer.zero_grad()
            
            if use_amp:
                with torch.amp.autocast('cuda'):
                    outputs = model(inputs)
                    loss = (criterion_raw(outputs, targets) * masks).sum() / (masks.sum() + 1e-8)
                
                scaler.scale(loss).backward()
                
                # ★勾配クリッピング (Gradient Clipping)
                # モデルが大きい時のエラー防止策。勾配が爆発するのを防ぎます。
                scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                
                scaler.step(optimizer)
                scaler.update()
            else:
                outputs = model(inputs)
                loss = (criterion_raw(outputs, targets) * masks).sum() / (masks.sum() + 1e-8)
                loss.backward()
                
                # ★勾配クリッピング (通常版)
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                
                optimizer.step()
            
            train_loss += loss.item()
            pbar.set_postfix({'loss': f"{loss.item():.6f}"})
            
        avg_train_loss = train_loss / len(train_loader)
        
        # Valid
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for inputs, targets, masks, _ in val_loader:
                inputs, targets, masks = inputs.to(device), targets.to(device), masks.to(device)
                if use_amp:
                    with torch.amp.autocast('cuda'):
                        outputs = model(inputs)
                        loss = (criterion_raw(outputs, targets) * masks).sum() / (masks.sum() + 1e-8)
                else:
                    outputs = model(inputs)
                    loss = (criterion_raw(outputs, targets) * masks).sum() / (masks.sum() + 1e-8)
                val_loss += loss.item()
        
        avg_val_loss = val_loss / len(val_loader)
        
        print(f"Result: [Epoch {current_epoch}] Train Loss: {avg_train_loss:.6f} | Val Loss: {avg_val_loss:.6f} | LR: {optimizer.param_groups[0]['lr']:.1e}")
        
        scheduler.step(avg_val_loss)
        
        if avg_val_loss < best_loss:
            print(f"score improved: {best_loss:.6f} --> {avg_val_loss:.6f}")
            best_loss = avg_val_loss
            patience_counter = 0
            torch.save(model.state_dict(), os.path.join(SAVE_DIR, "best_resnet1d_unet.pth"))
        else:
            patience_counter += 1
            print(f"No improvement. Patience: {patience_counter}/{PATIENCE}")
            if patience_counter >= PATIENCE:
                print("Early Stopping!")
                break
                
    print("All Finished!")

if __name__ == "__main__":
    run_training()

Using Device: cuda
Dataset Init...
Scanning files in C:\Users\fujiw\OneDrive\デスクトップ\ECG_ResNet\stage2...
Found 4395 files. Loading into RAM...


Pre-loading: 100%|█████████████| 4395/4395 [02:15<00:00, 32.48it/s]


Loaded 4395 samples.
Scanning files in C:\Users\fujiw\OneDrive\デスクトップ\ECG_ResNet\stage2...
Found 490 files. Loading into RAM...


Pre-loading: 100%|███████████████| 490/490 [00:19<00:00, 24.94it/s]


Loaded 490 samples.
Training Started (Large Model)...

==================== Epoch 1/200 ====================


...

Result: [Epoch 108] Train Loss: 0.002415 | Val Loss: 0.009186 | LR: 1.0e-08
No improvement. Patience: 18/20

==================== Epoch 109/200 ====================


Training   (Epoch 109): 100%|█| 138/138 [00:09<00:00, 15.33it/s, lo


Result: [Epoch 109] Train Loss: 0.002427 | Val Loss: 0.009204 | LR: 1.0e-08
No improvement. Patience: 19/20

==================== Epoch 110/200 ====================


Training   (Epoch 110): 100%|█| 138/138 [00:09<00:00, 15.29it/s, lo


Result: [Epoch 110] Train Loss: 0.002401 | Val Loss: 0.009215 | LR: 1.0e-08
No improvement. Patience: 20/20
Early Stopping!
All Finished!


Using Device: cuda
Dataset Init...
Scanning files in C:\Users\fujiw\OneDrive\デスクトップ\ECG_ResNet\stage2...
Found 3516 files. Loading into RAM...


Pre-loading: 100%|█████████████| 3516/3516 [00:56<00:00, 62.28it/s]


Loaded 3516 samples. (Augmentation: True)
Scanning files in C:\Users\fujiw\OneDrive\デスクトップ\ECG_ResNet\stage2...
Found 392 files. Loading into RAM...


Pre-loading: 100%|███████████████| 392/392 [00:08<00:00, 47.60it/s]


Loaded 392 samples. (Augmentation: False)
Training Started (Large Model + Augmentation)...

==================== Epoch 1/300 ====================


...

Result: [Epoch 2] Train Loss: 0.006304 | Val Loss: 0.002026 | LR: 1.0e-03
No improvement. Patience: 1/20

==================== Epoch 3/300 ====================


Training   (Epoch 3): 100%|█| 110/110 [00:14<00:00,  7.81it/s, loss


Result: [Epoch 3] Train Loss: 0.006376 | Val Loss: 0.001781 | LR: 1.0e-03
score improved: 0.001994 --> 0.001781

==================== Epoch 4/300 ====================


Training   (Epoch 4):  18%|▏| 20/110 [00:02<00:11,  7.61it/s, loss=


KeyboardInterrupt: 